### Driver Setup

In [4]:
# driver setup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from fake_useragent import UserAgent


ua = UserAgent()


def setup_driver():
    options = Options()
    options.add_argument("--headless=new") 
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080")
    options.add_argument(f"user-agent={ua.random}")

    driver = webdriver.Chrome(
        service=ChromeService(ChromeDriverManager().install()),
        options=options
        )
    return driver


def close_promo_bar(driver, timeout=10):
    try:
        # Wait for the close button to be present AND clickable
        close_button = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".promo-bar.new-seller .rb.close.dismiss"))
        )
        close_button.click()
        print("Promo bar closed successfully.")
    except TimeoutException:
        print("Promo bar close button not found or not clickable within timeout.")
    except Exception as e:
        print(f"Error closing promo bar: {e}")


def driver_teardown(driver):
    driver.quit()

### Scrape Auction URLs

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import csv
from datetime import datetime
import os


from driver_setup import close_promo_bar


def wait_for_pagination(driver, timeout:int=10):
    try:
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".paginator"))
        )
        print("✅ Pagination loaded (auctions likely loaded).")
    except TimeoutException:
        print("⚠️ Pagination not found. Proceeding anyway...")


def extract_auction_urls(driver, max_pages:int=2, timeout:int=30):
    """
    Scrapes auction URLs from carsandbids.com/past-auctions/.
    
    Args:
        driver: Selenium WebDriver instance.
        max_pages (int): Max number of pages to scrape. If None, scrape all.
        timeout (int): Timeout for WebDriverWait.
    Returns:
        list: All scraped auction URLs.
    """
    driver.get('https://carsandbids.com/past-auctions/')
    close_promo_bar(driver)


    auction_urls = []
    current_page = 1

    while True:
        print(f"Scraping page {current_page}...")

        try:
            # wait for auctions to load
            WebDriverWait(driver, timeout).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".auction-item")
            ))

            # extract urls from  current page
            auction_links = driver.find_elements(By.CSS_SELECTOR, ".auction-item .auction-title a[href]")
            auction_urls.extend([link.get_attribute("href") for link in auction_links])
            print(f"Added {len(auction_links)} URLs (Total: {len(auction_urls)})")


        except TimeoutException:
            print("No auctions found on page.")
            break
        except Exception as e:
            print("Error scraping auction urls.")
            print(e)
            break

        # check if it has scraped max_pages
        if max_pages and current_page >= max_pages:
            print(f"Reached max pages ({max_pages}). Stopping.")
            break

        # else go to the next page
        try:
            next_button = WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "li.arrow.next button"))
            )
            next_button.click()
            current_page += 1
            time.sleep(5)
        except TimeoutException:
            print("No more pages (or pagination button not clickable).")
            break
        except NoSuchElementException:
            print("Next button not found.")
            break
        except Exception as e:
            print(f"Error navigating to next page: {e}")
            break


    return auction_urls     

In [3]:
driver = setup_driver() # start driver
urls = extract_auction_urls(driver)
driver_teardown(driver) # close driver

urls

Promo bar closed successfully.
Scraping page 1...
Added 50 URLs (Total: 50)
Scraping page 2...
Added 50 URLs (Total: 100)
Reached max pages (2). Stopping.


['https://carsandbids.com/auctions/9W7bpAd5/2024-lotus-emira-v6-first-edition',
 'https://carsandbids.com/auctions/3OWAz7ay/2015-mercedes-amg-c63-s-sedan',
 'https://carsandbids.com/auctions/rNDZyG43/2004-bmw-m3-coupe',
 'https://carsandbids.com/auctions/KmJEMpVd/1994-land-rover-defender-90',
 'https://carsandbids.com/auctions/KDYqMvpa/1974-fiat-x1-9',
 'https://carsandbids.com/auctions/KmV46pAe/2015-porsche-cayenne-s-e-hybrid',
 'https://carsandbids.com/auctions/9lQboD8g/2023-bmw-m550i-xdrive',
 'https://carsandbids.com/auctions/9XpXPPmO/1998-mercedes-benz-s500',
 'https://carsandbids.com/auctions/KYpXyXXA/2016-range-rover-hse',
 'https://carsandbids.com/auctions/30nbbYwj/2018-jaguar-f-type-r-dynamic-coupe',
 'https://carsandbids.com/auctions/36BxDVB3/2003-porsche-boxster-s',
 'https://carsandbids.com/auctions/3gG6GzN8/2021-bmw-x5-m',
 'https://carsandbids.com/auctions/9n2Gjokv/1997-suzuki-carry-truck-4x4',
 'https://carsandbids.com/auctions/rkpg6waO/2024-tesla-model-y-long-range',
 '

### Scrape Auction

In [6]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import csv
from datetime import datetime
import os
import json


from driver_setup import close_promo_bar


def scrape_auction_data(driver, url:str, timeout:int = 30) -> dict:
    """
    Scrapes detailed information from a single auction page.
    
    Args:
        url: URL of the auction page
        driver: Selenium WebDriver instance
        timeout: Maximum wait time for elements
        
    Returns:
        Dictionary containing all scraped auction details
    """
    driver.get(url)
    close_promo_bar(driver)

    auction_data = {
        'auction_url': url,
        'auction_title': None,
        'auction_subtitle': None,
        'auction_stats':{
            'reserve_status': None,
            'auction_status': None,
            'highest_bid_value': None,
            'buyer_username': None,
            'seller_username': None,
            'bid_count': None,
            'view_count': None,
            'watcher_count': None,
            'auction_date': None,
            'bids':[]
            
        },
        'auction_quick_facts': {
            'Make': None,
            'Model': None,
            'Mileage': None,
            'VIN': None,
            'Title Status': None,
            'Location': None,
            'Seller': None,
            'Engine': None,
            'Drivetrain': None,
            'Transmission': None,
            'Body Style': None,
            'Exterior Color': None,
            'Interior Color': None,
            'Seller Type': None
        },
        'dougs_take': None,
        'auction_highlights': {
            'description': None,
            'bullet_points': []
        },
        'known_flaws': [],
        'service_history': {
            'description': None,
            'items': []
        },
        'included_items': [],
        'ownership_history': None,
        'seller_notes': [],
        'auction_videos': []
    }

    try:
        # Wait for main content to load
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".auction-title"))
        )
        # Extract title
        title_element = driver.find_element(By.CSS_SELECTOR, ".auction-title h1")
        auction_data['auction_title'] = title_element.text.strip()
        
        # Extract subtitle
        subtitle_element = driver.find_element(By.CSS_SELECTOR, ".d-md-flex.justify-content-between.flex-wrap h2")
        auction_data['auction_subtitle'] = subtitle_element.text.strip()
        
        # Extract reserve status
        reserve_element = driver.find_element(By.CSS_SELECTOR, "#auction-jump h3 span")
        auction_data['auction_stats']['reserve_status'] = 'Reserve' if 'Reserve' in reserve_element.text else 'No Reserve'
        
        # Extract auction status and final bid
        status_container = driver.find_element(By.CSS_SELECTOR, ".current-bid.ended")
        
        if 'cancelled' in status_container.get_attribute("class"):
            auction_data['auction_stats']['auction_status'] = 'Canceled'
        else:
            status_header = status_container.find_element(By.CSS_SELECTOR, "h4").text
            if 'Sold to' in status_header:
                auction_data['auction_stats']['auction_status'] = 'Sold'
                auction_data['auction_stats']['buyer_username'] = status_container.find_element(By.CSS_SELECTOR, ".username .user").text
            elif 'Reserve not met' in status_header:
                auction_data['auction_stats']['auction_status'] = 'Reserve Not Met'
            
            # Extract final bid amount
            bid_value = status_container.find_element(By.CSS_SELECTOR, ".bid-value").text
            auction_data['auction_stats']['highest_bid_value'] = bid_value.replace('$', '').strip()

        # Extract statistics from the stats ul
        stats_section = driver.find_element(By.CSS_SELECTOR, "ul.stats")
        
        # Seller information
        seller_element = stats_section.find_element(By.CSS_SELECTOR, "li.seller .user")
        auction_data['auction_stats']['seller_username'] = seller_element.text.strip()
        
        # Other stats
        stats_items = stats_section.find_elements(By.CSS_SELECTOR, "li:not(.seller)")
        for item in stats_items:
            label = item.find_element(By.CSS_SELECTOR, ".th").text.strip()
            value = item.find_element(By.CSS_SELECTOR, ".td").text.strip()
            
            if label == "Ended":
                auction_data['auction_stats']['auction_date'] = value
            elif label == "Bids":
                auction_data['auction_stats']['bid_count'] = int(value.replace(',', ''))
            elif label == "Views":
                auction_data['auction_stats']['view_count'] = int(value.replace(',', ''))
            elif label == "Watching":
                auction_data['auction_stats']['watcher_count'] = int(value.replace(',', ''))

        # Process auction quick facts
        # Wait for quick facts section to load
        try:
            WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".quick-facts"))
            )
            # Extract quick facts
            quick_facts = driver.find_element(By.CSS_SELECTOR, ".quick-facts")
            
            # Process first definition list
            first_dl = quick_facts.find_elements(By.CSS_SELECTOR, "dl")[0]
            items = first_dl.find_elements(By.CSS_SELECTOR, "dt")
            for item in items:
                label = item.text.strip().lower().replace(" ", "_")
                dd = item.find_element(By.XPATH, "./following-sibling::dd[1]")
                
                if label == "make":
                    auction_data['auction_quick_facts']['Make'] = dd.find_element(By.CSS_SELECTOR, "a").text.strip()
                elif label == "model":
                    auction_data['auction_quick_facts']['Model'] = dd.find_element(By.CSS_SELECTOR, "a").text.strip()
                elif label == "mileage":
                    auction_data['auction_quick_facts']['Mileage'] = dd.text.strip()
                elif label == "vin":
                    auction_data['auction_quick_facts']['VIN'] = dd.text.strip()
                elif label == "title_status":
                    auction_data['auction_quick_facts']['Title Status'] = dd.text.strip()
                elif label == "location":
                    auction_data['auction_quick_facts']['Location'] = dd.text.strip()
                elif label == "seller":
                    auction_data['auction_quick_facts']['Seller'] = dd.find_element(By.CSS_SELECTOR, ".user").text.strip()

            # Process second definition list
            second_dl = quick_facts.find_elements(By.CSS_SELECTOR, "dl")[1]
            items = second_dl.find_elements(By.CSS_SELECTOR, "dt")
            for item in items:
                label = item.text.strip().lower().replace(" ", "_")
                dd = item.find_element(By.XPATH, "./following-sibling::dd[1]")
                
                if label == "engine":
                    auction_data['auction_quick_facts']['Engine'] = dd.text.strip()
                elif label == "drivetrain":
                    auction_data['auction_quick_facts']['Drivetrain'] = dd.text.strip()
                elif label == "transmission":
                    auction_data['auction_quick_facts']['Transmission'] = dd.text.strip()
                elif label == "body_style":
                    auction_data['auction_quick_facts']['Body Style'] = dd.text.strip()
                elif label == "exterior_color":
                    auction_data['auction_quick_facts']['Exterior Color'] = dd.text.strip()
                elif label == "interior_color":
                    auction_data['auction_quick_facts']['Interior Color'] = dd.text.strip()
                elif label == "seller_type":
                    auction_data['auction_quick_facts']['Seller Type'] = dd.text.strip()

        except NoSuchElementException:
            print('Auction quick facts not found')
        except Exception as e:
            print(e)
            pass

        # Extract Doug's Take
        try:
            dougs_section = driver.find_element(By.CSS_SELECTOR, ".detail-section.dougs-take")
            auction_data['dougs_take'] = dougs_section.find_element(
                By.CSS_SELECTOR, ".detail-body p").text.strip()
        except NoSuchElementException:
            print("Doug's take not found")

        # Extract Highlights
        try:
            highlights_section = driver.find_element(By.CSS_SELECTOR, ".detail-section.detail-highlights")
            highlights_body = highlights_section.find_element(By.CSS_SELECTOR, ".detail-body")
            
            # Get description paragraph
            try:
                auction_data['auction_highlights']['description'] = highlights_body.find_element(
                    By.CSS_SELECTOR, "p").text.strip()
            except NoSuchElementException:
                pass
            
            # Get bullet points
            bullet_points = highlights_body.find_elements(By.CSS_SELECTOR, "ul li")
            auction_data['auction_highlights']['bullet_points'] = [
                point.text.strip() for point in bullet_points
                if point.text.strip()
            ]

        except NoSuchElementException:
            print('Auction highlights not found')

        # Extract Known Flaws
        try:
            flaws_section = driver.find_element(By.CSS_SELECTOR, ".detail-section.detail-known_flaws")
            flaws_items = flaws_section.find_elements(By.CSS_SELECTOR, ".detail-body li")
            auction_data['known_flaws'] = [item.text.strip() for item in flaws_items]
        except NoSuchElementException:
            print('Known flaws not found')


        # Extract Service History
        try:
            service_section = driver.find_element(By.CSS_SELECTOR, ".detail-section.detail-recent_service_history")
            auction_data['service_history']['description'] = service_section.find_element(
                By.CSS_SELECTOR, ".detail-body p").text.strip()
            service_items = service_section.find_elements(By.CSS_SELECTOR, ".detail-body li")
            auction_data['service_history']['items'] = [item.text.strip() for item in service_items]
        except NoSuchElementException:
            print('Service History not found')

         # Extract Included Items
        try:
            items_section = driver.find_element(By.CSS_SELECTOR, ".detail-section.detail-other_items")
            included_items = items_section.find_elements(By.CSS_SELECTOR, ".detail-body li")
            auction_data['included_items'] = [item.text.strip() for item in included_items]
        except NoSuchElementException:
            print("Included items not found")

        # Extract Ownership History
        try:
            history_section = driver.find_element(By.CSS_SELECTOR, ".detail-section.detail-ownership_history")
            auction_data['ownership_history'] = history_section.find_element(
                By.CSS_SELECTOR, ".detail-body p").text.strip()
        except NoSuchElementException:
            print('Ownership history not found')

        # Extract Seller Notes
        try:
            notes_section = driver.find_element(By.CSS_SELECTOR, ".detail-section.detail-seller_notes")
            notes_items = notes_section.find_elements(By.CSS_SELECTOR, ".detail-body li")
            auction_data['seller_notes'] = [item.text.strip() for item in notes_items]
        except NoSuchElementException:
            print('Seller notes not found')
        
        # Extract Video Links
        try:
            videos_section = driver.find_element(By.CSS_SELECTOR, ".detail-section.detail-videos")
            video_previews = videos_section.find_elements(By.CSS_SELECTOR, ".video-embed img.video-preview")
            auction_data['auction_videos'] = [
                img.get_attribute("src").split('/vi/')[1].split('/')[0] 
                for img in video_previews 
                if 'ytimg.com' in img.get_attribute("src")
            ]
        except NoSuchElementException:
            print('Auction videos not found')

        # bids
        try:
            # Wait for main content and click Bid History button
            WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".comments"))
            )
            
            # Click Bid History filter button
            try:
                bid_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-filter='4'][data-ga='bids']"))
                )
                driver.execute_script("arguments[0].click();", bid_button)
                time.sleep(2)  # Allow bids to load
            except Exception as e:
                print(f"Couldn't click bid history button: {str(e)}")
                return auction_data

            # Extract bid history
            bids = []
            bid_items = driver.find_elements(By.CSS_SELECTOR, ".thread li.bid")
            for bid in bid_items:
                try:
                    bid = bid.find_element(By.CSS_SELECTOR, ".bid-value").text.replace('$', '').replace(',', '')
                    bids.append(bid)
                    # bid_data = {
                    #     # 'bidder': bid.find_element(By.CSS_SELECTOR, ".user").text.strip(),
                        
                    #     # 'time': bid.find_element(By.CSS_SELECTOR, ".time").get_attribute("data-full"),
                    #     # 'is_verified': bool(bid.find_elements(By.CSS_SELECTOR, ".verified")),
                    #     # 'reputation': bid.find_element(By.CSS_SELECTOR, ".rep").text.replace('Reputation Icon', '').strip()
                    # }
                   
                except Exception as e:
                    print(f"Error parsing bid: {str(e)}")
                    continue
            auction_data['auction_stats']['bids']=bids

        except Exception as e:
            print(f"Error scraping bid history: {str(e)}")

    except TimeoutException:
        print(f"Timeout while scraping {url}")
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
    
    return auction_data


In [30]:
driver = setup_driver()

auctions_data = []

for url in urls[:3]:
    auction_data = scrape_auction_data(driver, url)
    auction_data['auction_date'] = auction_data['auction_stats']['auction_date']
    auctions_data.append(auction_data)

driver_teardown(driver)

import json
with open ("auction_test.json", "w") as f:
    json.dump(auctions_data, f, indent=3)

Promo bar closed successfully.


KeyboardInterrupt: 

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed


def one_url_driver(url):
    driver = setup_driver()
    data = scrape_auction_data(driver, url)
    driver_teardown(driver)
    return data


driver = setup_driver()
auctions_data = []

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(one_url_driver, url) for url in urls[:5]]
    for future in as_completed(futures):
        try:
            result = future.result()
            auctions_data.append(future.result())
        except Exception as e:
            print(f"Error: {e}")

driver_teardown(driver)
import json
with open ("auction_test.json", "w") as f:
    json.dump(auctions_data, f, indent=3)

Promo bar closed successfully.
Promo bar closed successfully.
Promo bar closed successfully.
Promo bar closed successfully.
Promo bar closed successfully.


### Upload to s3

### Parquet files in Bronze layer

In [8]:

import os
import io
import pandas as pd
import boto3

DBT_ENV_SECRET_AWS_ACCESS_KEY_ID = os.getenv("DBT_ENV_SECRET_AWS_ACCESS_KEY_ID")
DBT_ENV_SECRET_AWS_SECRET_ACCESS_KEY = os.getenv("DBT_ENV_SECRET_AWS_SECRET_ACCESS_KEY")
DBT_ENV_SECRET_AWS_REGION = os.getenv("DBT_ENV_SECRET_AWS_REGION")
DBT_ENV_S3_BUCKET = os.getenv("DBT_ENV_S3_BUCKET")

In [9]:
df = pd.DataFrame(auctions_data)
df['auction_date'] = pd.to_datetime(df['auction_date'], format="%b %d, %Y %I:%M %p")
df

,auction_url,auction_title,auction_subtitle,auction_stats,auction_quick_facts,dougs_take,auction_highlights,known_flaws,service_history,included_items,ownership_history,seller_notes,auction_videos,auction_date
0,https://carsandbids.com/auctions/9W7bpAd5/2024...,2024 Lotus Emira V6 First Edition,"INSPECTED ~2,300 Miles, 6-Speed Manual, 3.5-Li...","{'reserve_status': 'Reserve', 'auction_status'...","{'Make': 'Lotus', 'Model': 'Emira', 'Mileage':...",The Lotus Emira is a thrilling sports car that...,{'description': 'THIS… is a 2024 Lotus Emira V...,"[Chip on windshield, Creases on driver's seat ...",{'description': 'The attached Carfax history r...,"[2 keys and velvet key pouches, Owner's manual...",The seller reports that they purchased this Lo...,[],[],2025-10-31 23:07:00
1,https://carsandbids.com/auctions/rNDZyG43/2004...,2004 BMW M3 Coupe,"NO RESERVE 6-Speed Manual, S54 6-Cylinder, Pre...","{'reserve_status': 'No Reserve', 'auction_stat...","{'Make': 'BMW', 'Model': 'E46 M3', 'Mileage': ...",The E46 BMW M3 is a crowd favorite here at Car...,{'description': 'THIS... is a 2004 BMW M3 Coup...,"[Chips, scratches, and dings around the exteri...",{'description': 'The attached Carfax report sh...,"[1 key, Owner's manuals and booklets, Window s...",The seller reports that they purchased this M3...,[Due to the modifications performed to this M3...,[],2025-10-31 23:03:00
2,https://carsandbids.com/auctions/3OWAz7ay/2015...,2015 Mercedes-AMG C63 S Sedan,"503-hp Twin-Turbo V8, AMG Performance Seats, M...","{'reserve_status': 'Reserve', 'auction_status'...","{'Make': 'Mercedes-Benz', 'Model': 'C-Class AM...",The W205 Mercedes-AMG C63 is a thrilling sport...,{'description': 'THIS... is a 2015 Mercedes-AM...,[The attached Carfax vehicle history report in...,{'description': 'The attached Carfax vehicle h...,"[1 key, Available service records]",The seller reports that they purchased this C6...,"[There is a loan on this vehicle, and sale pro...",[],2025-10-31 23:04:00
3,https://carsandbids.com/auctions/KmJEMpVd/1994...,1994 Land Rover Defender 90,"TATC Defender Build, 5-Speed Manual, Left-Hand...","{'reserve_status': 'Reserve', 'auction_status'...","{'Make': 'Land Rover', 'Model': 'Defender', 'M...",We love the original Land Rover Defender here ...,{'description': 'THIS... is a 1994 Land Rover ...,[This Defender's odometer was reset to zero up...,{'description': 'The seller states that this D...,"[2 keys, Exmoor Trim soft top]",The seller is representing this Defender on be...,[],[],2025-10-31 22:58:00
4,https://carsandbids.com/auctions/KDYqMvpa/1974...,1974 Fiat X1/9,"NO RESERVE 1.5-Liter 4-Cylinder Swap, 5-Speed ...","{'reserve_status': 'No Reserve', 'auction_stat...","{'Make': 'Fiat', 'Model': 'X1/9', 'Mileage': '...",The Fiat X1/9 is a cool little sports car with...,"{'description': 'THIS... is a 1974 Fiat X1/9, ...","[Chips, scratches, scuffs, and other exterior ...",{'description': 'The seller reports that the f...,"[2 keys, Owner's manual, Service records, Spar...",The seller reportedly purchased this X1/9 in A...,[Vintage vehicles may not carry emissions equi...,[],2025-10-31 22:53:00


In [11]:
from datetime import datetime, timedelta

key = (datetime.now().date() - timedelta(days=0)).isoformat()
key

s3_client = boto3.client(
    's3',
    aws_access_key_id=DBT_ENV_SECRET_AWS_ACCESS_KEY_ID,
    aws_secret_access_key=DBT_ENV_SECRET_AWS_SECRET_ACCESS_KEY,
    region_name=DBT_ENV_SECRET_AWS_REGION

)

buffer = io.BytesIO()
df.to_parquet(buffer, engine="pyarrow", compression="snappy", index=False)
buffer.seek(0)

s3_client.put_object(
    Bucket = DBT_ENV_S3_BUCKET, 
    Key = f"carsnbids/{key}.parquet", 
    Body = buffer.getvalue()
)

{'ResponseMetadata': {'RequestId': '4GDZT5JM4N5EAZ2C',
  'HostId': '4Rp7c2+smiLgggFDvpQU1Fv+0tGXwtrJlKd/UphzS6NU7sknhWNJSm9d9uIUZqvce7SHSRFTcGE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '4Rp7c2+smiLgggFDvpQU1Fv+0tGXwtrJlKd/UphzS6NU7sknhWNJSm9d9uIUZqvce7SHSRFTcGE=',
   'x-amz-request-id': '4GDZT5JM4N5EAZ2C',
   'date': 'Sun, 02 Nov 2025 15:02:29 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"4903ffd492e3550ec8ed3bc96dde2015"',
   'x-amz-checksum-crc32': 'uTxNEQ==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"4903ffd492e3550ec8ed3bc96dde2015"',
 'ChecksumCRC32': 'uTxNEQ==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

### main